In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import torch

# if torch.cuda.is_available():

#   device_count = torch.cuda.device_count()
#   print("device_count: {}".format(device_count))

#   for device_num in range(device_count):
#     print("device {} capability {}".format(
#     device_num,
#     torch.cuda.get_device_capability(device_num)))
#     print("device {} name {}".format(
#     device_num, torch.cuda.get_device_name(device_num)))
# else:
#   print("no cuda device")

device_count: 1
device 0 capability (8, 0)
device 0 name NVIDIA A100-SXM4-40GB


In [ ]:
# import locale
# def getpreferredencoding(do_setlocale = True):
#     return "UTF-8"

In [1]:
!pip install -q accelerate==0.26.1 peft==0.8.2 bitsandbytes==0.42.0 transformers==4.37.2 langchain faiss-gpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
!pip -q install gradio==3.45.0 --use-deprecated=legacy-resolver typing_extensions --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.3/298.3 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.7 MB/s eta 0:00:00


In [3]:
import torch
import time
import gradio as gr
from peft import (PeftModel,
                  AutoPeftModelForCausalLM)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer
    )
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
import faiss

In [4]:
compute_dtype = getattr(torch, 'float16')

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [5]:
peft_model = AutoPeftModelForCausalLM.from_pretrained("uine/practice-fine-tuning",
                                                      quantization_config=quant_config,
                                                      device_map={"": 0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/164M [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("uine/practice-fine-tuning")

In [7]:
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

embedding_size = 768
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = hf.embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})
retriever = vectorstore.as_retriever(search_kwargs=dict(k=2))
memory = VectorStoreRetrieverMemory(retriever=retriever, return_docs=False)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
# streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

# s = '나 너무 우울해'
# history = memory.load_memory_variables({"query": s})["history"]
# ss=''

# for texts in history.split('\n'):
#   ss += texts + ' '

# context = ss + '질문: ' + s

# SYSTEM_PROMPT = """
# <s>[INST] <<SYS>>You are a counselor like friend who empathizes, encourages, and helps person who is anxious or depressed. Refer to the context of the conversation to answer current conversation. At the end of your answer, please ask questions related to the content so far. You must complete your answer in three sentences. Be sure not to repeat the same answer.<</SYS>>
# """
# conversation = [{'role': 'user', 'content': SYSTEM_PROMPT + context + '[/INST]'}]
# inputs = tokenizer.apply_chat_template(
# conversation,
# tokenize=True,
# add_generation_prompt=False,
# return_tensors='pt'
# ).to("cuda")
# _ = peft_model.generate(inputs, streamer=streamer, max_new_tokens=150, repetition_penalty=1.2)
# response = tokenizer.decode(_[0][len(inputs[0]):])

# memory.save_context(
#         {"질문": s},
#         {"답변": response},
#     )

네, 그런 날도 있죠. 특별히 어떤 일로 인해 우울한 기분을 느낀다고 말씀하시겠어요? 그리고 그것이 어떻게 당신의 일상에 영향을 미치고 계신지 알려주실 수 있을까요?


In [13]:
def res(message: str, history: list) -> str:

    history = memory.load_memory_variables({"query": message})["history"]
    ss=''

    for texts in history.split('\n'):
      ss += texts + ' '

    context = ss + '질문: ' + message

    SYSTEM_PROMPT = """
    <s>[INST] <<SYS>>You are a counselor like friend who empathizes, encourages, and helps person who is anxious or depressed. You must complete your answer in three sentences. Be sure not to repeat the same answer.<</SYS>>
    """

    # SYSTEM_PROMPT = """
    # <s>[INST] <<SYS>>You are a counselor like friend who empathizes, encourages, and helps person who is anxious or depressed. Refer to the context of the conversation to answer current conversation. At the end of your answer, please ask questions related to the content so far. You must complete your answer in three sentences. Be sure not to repeat the same answer.<</SYS>>
    # """

    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    conversation = [{'role': 'user', 'content': SYSTEM_PROMPT + context + '[/INST]'}]
    inputs = tokenizer.apply_chat_template(
    conversation,
    tokenize=True,
    add_generation_prompt=False,
    return_tensors='pt'
    ).to("cuda")
    _ = peft_model.generate(inputs, streamer=streamer, max_new_tokens=250, repetition_penalty=1.2)
    response = tokenizer.decode(_[0][len(inputs[0]):])

    memory.save_context(
            {"질문": message},
            {"답변": response},
        )

    print("과거 대화:", ss, '\n')
    print("사용자 입력:", message, '\n')
    print("멘토스 답변:", response, '\n')

    for i in range(len(response)):
        time.sleep(0.03)
        yield response[:i+1]

In [ ]:
gr.ChatInterface(
        fn=res,
        textbox=gr.Textbox(placeholder="고민을 얘기해주세요🙌", container=False, scale=1),
        title="멘토스(Mental Mate Talk on Support)",
        description="멘토스는 당신의 고민을 들어주며 격려해주는 상담친구에요😊",
        theme="soft",
        examples=[["나 우울해"], ["너무 짜증나"], ["사는게 쉽지않아"]],
        retry_btn="다시보내기 ↩",
        undo_btn="이전챗 삭제 ❌",
        clear_btn="전챗 삭제 💫"
).queue().launch(debug=True, share=True)